In [14]:
import sympy as sp
from sympy import Matrix, symbols, Symbol

def copy_images_bottom_channel_sympy(image, J):
    # Assume 'image' is a sympy Matrix representing a grayscale image
    # 'J' is the scale factor

    # Original dimensions
    orig_rows, orig_cols = image.shape

    # New dimensions
    new_rows = orig_rows * J
    new_cols = orig_cols * J

    # Creating a new matrix for the scaled image
    scaled_image = sp.zeros(new_rows, new_cols)

    # Implementing nearest neighbor interpolation
    for i in range(new_rows):
        for j in range(new_cols):
            # Find the nearest pixel from the original image
            orig_i = i // J
            orig_j = j // J
            scaled_image[i, j] = image[orig_i, orig_j]

    return scaled_image

def to_density_matrix_sympy(vector):
    # Initialize a list to store the resulting matrices
    # Convert the row vector to a column vector for the outer product
    column_vector = vector.T
    # Compute the outer product
    density_matrix = column_vector * column_vector.T
    return sp.Matrix(density_matrix)

# Example usage
# Input matrix, where each row is a vector
I = 4  # Initial image size (e.g., 4x4)
N = I
O = I//2  # Output image size (e.g., 2x2)
J = 8  # Number of quantum channels

# variables = sp.symbols(['x{}__{}'.format(i, j) for i in range(1, 5) for j in range(1, 5)])
x = sp.Matrix([[Symbol(f'x_{i+1},{j+1}') for j in range(N)] for i in range(N)])
vector = sp.Matrix([Symbol(f'x_{i+1},{j+1}') for i in range(N) for j in range(N)])
# display(vector)
# vector = sp.Matrix(variables)
# display(vector)
# vector = sp.Matrix([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
density_matrix_image = vector * vector.T

# Call the function
channel_image = copy_images_bottom_channel_sympy(density_matrix_image, J)
# channel_image = copy_images_bottom_channel_sympy_stride(density_matrix_image, J)
# display(channel_image)

In [9]:

import torch
device = torch.device("cpu")
import torch

import sympy as sp

def Pooling_2D_Projector_3D_SymPy(I, O, J):
    """ This function mimics the effect of a HW preserving pooling layer on half
    the remaining qubits using SymPy for symbolic computation. We suppose that the input image is square.
    Args:
        - I: size of the input image
        - O: size of the output image (we suppose O = I//2)
        - J: number of channels or depth, specific to quantum state representation.
    Output:
        - Projector: projector corresponding to all cases of measurement. Its
        dimension is (k, O**2*J, I**2*J) with k the number of cases of measurement.
    """
    # Number of matrices:
    Number_of_matrices, index = 1 + 2*O + O**2, 0
    # Creating a list of zero matrices
    Projectors = [sp.zeros(O*O*J, I*I*J) for _ in range(Number_of_matrices)]

    # We consider the case where all measured qubits are on state |0>:
    for i in range(O):
        for j in range(O):
            for c in range(J):
                Projectors[index][(i*O+j)*J+c, ((i*2+1)*I+(j*2+1))*J+c] = 1
    index += 1

    # We consider the case where we measured 2 qubits in state |1>:
    for i in range(O):
        for j in range(O):
            for c in range(J):
                Projectors[index][(i*O+j)*J+c, ((i*2)*I+(j*2))*J+c] = 1
            index += 1

    # We finally consider the case where we measured only one qubit in state |1>:
    # If we measure a qubit in |1> in the line register
    for i in range(O):
        for j in range(O):
            for c in range(J):
                Projectors[index][(i*O+j)*J+c, ((2*i)*I + 2*j+1)*J+c] = 1
        index += 1

    # If we measure a qubit in |1> in the column register
    for j in range(O):
        for i in range(O):
            for c in range(J):
                Projectors[index][(i*O+j)*J+c, ((2*i+1)*I + 2*j)*J+c] = 1
        index += 1

    return Projectors

class Pooling_2D_density_3D:
    """ This module describes the effect of the Pooling on the QCNN architecture while
    simulating states as density operators. """
    def __init__(self, I, O, J):
        """ We suppose that the input image is square. """
        self.Projectors = Pooling_2D_Projector_3D_SymPy(I, O, J)
        self.O = O
        self.J = J

    def forward(self, input):
        """ This module forwards a matrix made of each pure state weighted by their
        probabilities that describe the output mixed state from the pooling layer.
        Arg:
            - input: a list of SymPy matrices representing the initial input state (density matrices).
            Each matrix dimension is (I**2*J, I**2*J).
        Output:
            - a list of SymPy matrices that represent the output mixed
            state with dimension (O**2*J, O**2*J).
        """
        mixed_state_density_matrix = sp.zeros((self.O**2)*self.J, (self.O**2)*self.J)
        for p in self.Projectors:
            mixed_state_density_matrix += p * input* p.transpose()
        return mixed_state_density_matrix


# matrix = sp.Matrix(n, n, lambda i, j: sp.symbols(f'x{i+1},{j+1}'))
pooling_module = Pooling_2D_density_3D(I, O, J)
output_density_matrix = pooling_module.forward(channel_image)
display(output_density_matrix)

Matrix([
[x_1,1**2 + x_1,2**2 + x_2,1**2 + x_2,2**2, x_1,1**2 + x_1,2**2 + x_2,1**2 + x_2,2**2,                 x_1,2*x_1,4 + x_2,2*x_2,4,                 x_1,2*x_1,4 + x_2,2*x_2,4,                 x_2,1*x_4,1 + x_2,2*x_4,2,                 x_2,1*x_4,1 + x_2,2*x_4,2,                               x_2,2*x_4,4,                               x_2,2*x_4,4],
[x_1,1**2 + x_1,2**2 + x_2,1**2 + x_2,2**2, x_1,1**2 + x_1,2**2 + x_2,1**2 + x_2,2**2,                 x_1,2*x_1,4 + x_2,2*x_2,4,                 x_1,2*x_1,4 + x_2,2*x_2,4,                 x_2,1*x_4,1 + x_2,2*x_4,2,                 x_2,1*x_4,1 + x_2,2*x_4,2,                               x_2,2*x_4,4,                               x_2,2*x_4,4],
[                x_1,2*x_1,4 + x_2,2*x_2,4,                 x_1,2*x_1,4 + x_2,2*x_2,4, x_1,3**2 + x_1,4**2 + x_2,3**2 + x_2,4**2, x_1,3**2 + x_1,4**2 + x_2,3**2 + x_2,4**2,                               x_2,4*x_4,2,                               x_2,4*x_4,2,                 x_2,3*x_4,3 + x_2,4*x_4,

In [15]:
def Pooling_channel_Projector_3D_sympy(I, O, J, L):
    """ This function mimics the effect of a HW preserving 3D channel pooling layer on half
    the remaining qubits. We suppose that the input image is square.
    Args:
        - I: size of the input image
        - O: size of the output image (we suppose O = I//2 for now)
    Output:
        - Projector: list of projectors corresponding to all cases of measurement. Each projector
        is a SymPy matrix with dimension (O**2 * L, I**2 * J).
    """
    # Number of matrices:
    Number_of_matrices = (1 + 2 * O + O ** 2) * J
    index = 0

    # Initialize the list of projectors
    Projectors = [sp.zeros(O * O * L, I * I * J) for _ in range(Number_of_matrices)]

    # We consider the case where all measured qubits are on state |0>: O=2, J=4
    for c in range(J):
        for i in range(O):
            for j in range(O):
                Projectors[index][((i * O + j) * J + c) // 2, ((i * 2 + 1) * I + (j * 2 + 1)) * J + c] = 1
        # if c % 2 == 1:
        index += 1

    # We consider the case where we measured 2 qubits in state |1>:
    for i in range(O):
        for j in range(O):
            for c in range(J):
                Projectors[index][((i * O + j) * J + c) // 2, ((i * 2) * I + (j * 2)) * J + c] = 1
                # if c % 2 == 1:
                index += 1

    # We finally consider the case where we measured only one qubit in state |1>:
    # If we measure a qubit in |1> in the line register
    for i in range(O):
        for c in range(J):
            for j in range(O):
                Projectors[index][((i * O + j) * J + c) // 2, ((2 * i) * I + 2 * j + 1) * J + c] = 1
            # if c % 2 == 1:
            index += 1

    # If we measure a qubit in |1> in the column register
    for j in range(O):
        for c in range(J):
            for i in range(O):
                Projectors[index][((i * O + j) * J + c) // 2, ((2 * i + 1) * I + 2 * j) * J + c] = 1
            # if c % 2 == 1:
            index += 1

    return Projectors

class Pooling_channel_density_3D_sympy:
    """ This module describe the effect of the Pooling on the QCNN architecture while
    simulating states as density operators. """

    def __init__(self, I, O, J):
        """ We suppose that the input image is square. """
        self.Projectors = Pooling_channel_Projector_3D_sympy(I, O, J, J//2)
        self.O = O
        self.J = J // 2

    def forward(self, input):
        """ This module forwards a tensor made of each pure state weighted by their
        probabilities that describe the output mixed state from the pooling layer.
        Args:
            - input: a SymPy matrix representing the initial input state (density matrix).
            The matrix has dimension (I**2, I**2).
        Output:
            - a SymPy matrix that represents the output mixed state with dimension (O**2, O**2).
        """
        mixed_state_density_matrix = sp.zeros((self.O ** 2) * self.J, (self.O ** 2) * self.J)

        for p in self.Projectors:
            projector = sp.Matrix(p)
            mixed_state_density_matrix += projector * input * projector.T

        return mixed_state_density_matrix



pooling_module = Pooling_channel_density_3D_sympy(I, O, J)
output_density_matrix = pooling_module.forward(channel_image)
display(output_density_matrix)

Matrix([
[2*x_1,1**2 + 2*x_1,2**2 + 2*x_2,1**2 + 2*x_2,2**2,                                                 0,                                                 0,                                                 0,                     2*x_1,2*x_1,4 + 2*x_2,2*x_2,4,                                                 0,                                                 0,                                                 0,                     2*x_2,1*x_4,1 + 2*x_2,2*x_4,2,                                                 0,                                                 0,                                                 0,                                     2*x_2,2*x_4,4,                                                 0,                                                 0,                                                 0],
[                                                0, 2*x_1,1**2 + 2*x_1,2**2 + 2*x_2,1**2 + 2*x_2,2**2,                                                 0,                    

In [16]:
pooling_module2 = Pooling_channel_density_3D_sympy(I//2, O//2, J//2)
output_density_matrix = pooling_module2.forward(output_density_matrix)
display(output_density_matrix)

Matrix([
[4*x_1,1**2 + 4*x_1,2**2 + 4*x_1,3**2 + 4*x_1,4**2 + 4*x_2,1**2 + 4*x_2,2**2 + 4*x_2,3**2 + 4*x_2,4**2 + 4*x_3,1**2 + 4*x_3,2**2 + 4*x_3,3**2 + 4*x_3,4**2 + 4*x_4,1**2 + 4*x_4,2**2 + 4*x_4,3**2 + 4*x_4,4**2,                                                                                                                                                                                                             0],
[                                                                                                                                                                                                            0, 4*x_1,1**2 + 4*x_1,2**2 + 4*x_1,3**2 + 4*x_1,4**2 + 4*x_2,1**2 + 4*x_2,2**2 + 4*x_2,3**2 + 4*x_2,4**2 + 4*x_3,1**2 + 4*x_3,2**2 + 4*x_3,3**2 + 4*x_3,4**2 + 4*x_4,1**2 + 4*x_4,2**2 + 4*x_4,3**2 + 4*x_4,4**2]])